Trivial NSE Solver
==
for a much better one see: https://cococubed.com/code_pages/nse.shtml

In [ ]:
import numpy as np
import matplotlib.pylab as plt
from scipy.optimize import fsolve

We want to solve for the NSE of neutrons, protons, and alpha particles.  A simple three species system. We have three dependent quantities that fully describe the system. 1. $\rho$, the total mass density; 2. $T$, the temperature of the matter; 3. $Y_p$ the proton fraction, which sets the ratio of protons to baryons in the system.

For ideal gases, the chemical potential can be determined as a combintation of the particle number density ($n$), the particle's quantum concentration ($n_Q$), and the temperature ($T$),

$$\mu_i = k_B T \ln (n_i/n_{Q,i})$$

we will define the $k_B T$ in units of MeV and just call it $T$ from now on. The number density of a species can be defined as the mass fraction of the species, $\rho_i = X_i \rho$, divided by the atomic mass of the species, so $n_i = X_i \rho_i / A_i$. Here, $X_i$ is the mass fraction of species $i$. The quantum concentration is given as

$$n_{Q,i} = \left(\frac{2 \pi T A_i m_\mathrm{amu} c^2 }{ (hc)^2} \right)^{3/2}$$

As of now, our three species system has three unknowns, that is the three different number densities (or equivilently the three chemical potentials).  We have two conditions to help eliminate these unknowns.  1. the sum of the mass fractions must be 1, i.e. $1 = X_n + X_p + X_\alpha$ and 2. the sum of the protons must be equal to the proton fraction, i.e. $Y_p = X_p + X_\alpha/2$.  

The final condition is that of NSE, i.e. that the chemical potentials of composite particles (like alpha particles, 2 neutrons and 2 protons) are related to the individual chemical potentials of the neutron and proton. 

$$\mu_i = Z_i \mu_p + (A_i-Z_i) \mu_n + B_i$$

where $Z_i$ is the atomic charge  In equilibrium the energy cost of producing a composite particle is simply the chemical potentials of the constitents plus, importantly, the binding energy of the composite particle.  

In [ ]:
#constants
mamuc2 = 931.494103 #MeV
mamu_grams =  1.66053907e-24 #grams
Navo = 6.02214076e23
kB = 1 #if T is in MeV, kT (an energy), will be in MeV
hc = 1.24e-10 #MeV cm

In [ ]:
A = np.array([1,1,4])
Z = np.array([0,1,2])
B = np.array([0,0,28.3]) #MeV

In [ ]:
#return the mass fraction X given a chemical potential (inverting the expression for mu above)
def X(xmu,xA,xrho,xT):
    nQ = (2*np.pi*xT*xA*mamuc2/hc**2)**(1.5)
    return xA/(xrho*Navo)*nQ*np.exp(xmu/xT)

#NSE condition for alpha particles:
def mualpha(mun,mup):
    return 2*mun+2*mup+28.3

#chemical potential given temperature (for the quantum concentration) and the number density of particles
def mu(T,n,A):
    nQ = (2*np.pi*T*A*mamuc2/hc**2)**(1.5)
    return T*np.log(n/nQ)

def check_state(mun,mup,mualpha,rho,T):
    Xp = X(mup,1,rho,T)
    Xn = X(mun,1,rho,T)
    Xa = X(mualpha,1,rho,T)
    print(Xp,Xn,Xa,Xp+Xn+Xa)

In [ ]:
def get_Xs(rho,T,yp):

    #start by assuming all N and P,
    npguess = yp*rho/mamu_grams
    nnguess = (1-yp)*rho/mamu_grams
    mupguess = mu(T,npguess,1.)
    munguess = mu(T,nnguess,1.)
    mualphaguess = mualpha(munguess,mupguess)

    if mualphaguess>munguess:
        #maybe lots of alpha?, the above guess will be too far off, assume lower densities for free nucleons
        if yp<0.5:
            npguess = min(0.01,yp)*rho/mamu_grams
            nnguess = 2.0*(0.5-yp)*rho/mamu_grams
            mupguess = mu(T,npguess,1.)
            munguess = mu(T,nnguess,1.)
            mualphaguess = mualpha(munguess,mupguess)
        elif yp>=0.5:
            nnguess = min(0.01,(1.0-yp))*rho/mamu_grams
            npguess = 2.0*(yp-0.499)*rho/mamu_grams
            mupguess = mu(T,npguess,1.)
            munguess = mu(T,nnguess,1.)
            mualphaguess = mualpha(munguess,mupguess)

    def LS(x):
        return [1-X(x[0],1,rho,T)- X(x[1],1,rho,T)-X(x[2],4,rho,T), #sum(X)=1
                yp-X(x[0],1,rho,T)-X(x[2],4,rho,T)/2., #sum(ZX/A)=Yp
                x[2]-mualpha(x[1],x[0])] #NSE for alphas

    values = fsolve(LS, [mupguess,munguess,mualphaguess])
    
    Xp = X(values[0],1,rho,T)
    Xn = X(values[1],1,rho,T)
    Xa = X(values[2],4,rho,T)
    
    return Xp,Xn,Xa

In [ ]:
rho=1e11
yp = 0.25
Ts = np.linspace(1,10,100)
Xps = np.zeros(100).reshape(100)
Xns = np.zeros(100).reshape(100)
Xalphas = np.zeros(100).reshape(100)
count=0
for T in Ts:
    (Xps[count],Xns[count],Xalphas[count]) = get_Xs(rho,T,yp)
    count += 1
    

In [ ]:
plt.plot(Ts,Xps,label="Xp")
plt.plot(Ts,Xns,label="Xn")
plt.plot(Ts,Xalphas,label="Xalpha")
plt.legend(frameon=False)
plt.xlabel("Temperature [MeV]")
plt.ylabel("Mass Fraction")
plt.title("NSE for log10rho="+str(np.log10(rho))+"; Yp="+str(yp))
plt.show()

In [ ]:
rho=1e11
T = 3.
yps = np.linspace(0.001,0.999,100)
Xps = np.zeros(100).reshape(100)
Xns = np.zeros(100).reshape(100)
Xalphas = np.zeros(100).reshape(100)
count=0
for yp in yps:
    (Xps[count],Xns[count],Xalphas[count]) = get_Xs(rho,T,yp)
    count += 1

In [ ]:
plt.plot(yps,Xps,label="Xp")
plt.plot(yps,Xns,label="Xn")
plt.plot(yps,Xalphas,label="Xalpha")
plt.legend(frameon=False)
plt.xlabel("Yp")
plt.ylabel("Mass Fraction")
plt.title("NSE for log10rho="+str(np.log10(rho))+"; T="+str(T))
plt.show()